In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import skimage.io
import skimage.filters
from skimage.filters import try_all_threshold
import matplotlib.image as mpimg
from scipy import ndimage as ndi
from skimage.transform import resize
import skimage.morphology
import skimage.feature
import math
from math import sqrt
from skimage.color import rgb2gray
from skimage import measure
from skimage import data, util

In [2]:
os.getcwd()

'/Users/mikemckenna/Desktop/image_analysis'

In [3]:
prefix = ['NT_48h', 'OGD_24h', 'OGD_48h']
#conditions = ['NT', 'OGD']
#time_points = ['24', '48']
regions = ['cortex', 'striatum', 'corpus_callosum']
images = 5

image_list = []

for pref in prefix:
    for region in regions:
        for num in range(1,images+1):
            image_list.append('{}_PI_{}_20x_images_XY{}.tif'.format(pref, region,num))

In [4]:
image_list

['NT_48h_PI_cortex_20x_images_XY1.tif',
 'NT_48h_PI_cortex_20x_images_XY2.tif',
 'NT_48h_PI_cortex_20x_images_XY3.tif',
 'NT_48h_PI_cortex_20x_images_XY4.tif',
 'NT_48h_PI_cortex_20x_images_XY5.tif',
 'NT_48h_PI_striatum_20x_images_XY1.tif',
 'NT_48h_PI_striatum_20x_images_XY2.tif',
 'NT_48h_PI_striatum_20x_images_XY3.tif',
 'NT_48h_PI_striatum_20x_images_XY4.tif',
 'NT_48h_PI_striatum_20x_images_XY5.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY1.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY2.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY3.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY4.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY5.tif',
 'OGD_24h_PI_cortex_20x_images_XY1.tif',
 'OGD_24h_PI_cortex_20x_images_XY2.tif',
 'OGD_24h_PI_cortex_20x_images_XY3.tif',
 'OGD_24h_PI_cortex_20x_images_XY4.tif',
 'OGD_24h_PI_cortex_20x_images_XY5.tif',
 'OGD_24h_PI_striatum_20x_images_XY1.tif',
 'OGD_24h_PI_striatum_20x_images_XY2.tif',
 'OGD_24h_PI_striatum_20x_images_XY3.tif',
 'OGD_24h_P

In [5]:
len(image_list)

45

In [8]:
output_data = pd.DataFrame(columns=['Image', 'DAPI count','DAPI intensity', 'DAPI area', 'PI count', 'PI intensity', 'PI area', 'PI intensity/DAPI count', 'PI w/in DAPI area', 'PI w/in DAPI/DAPI count', '%overlap PI/DAPI', '% cell death'])

In [9]:
output_data

,Image,DAPI count,DAPI intensity,DAPI area,PI count,PI intensity,PI area,PI intensity/DAPI count,PI w/in DAPI area,PI w/in DAPI/DAPI count,%overlap PI/DAPI,% cell death


In [10]:
for image in image_list:
    img = skimage.io.imread(image)
    DAPI = img[0]
    PI = img[1]
    #DAPI_filtered = ndi.gaussian_filter(DAPI, sigma=1)
    #PI_filtered = ndi.gaussian_filter(PI, sigma=1)
    DAPI_thresh = skimage.filters.threshold_otsu(DAPI)
    DAPI_binary = DAPI > DAPI_thresh
    PI_thresh = skimage.filters.threshold_otsu(PI)
    PI_binary = PI > PI_thresh
    #DAPI_dilation = skimage.morphology.binary_dilation(DAPI_binary)
    DAPI_filled_holes = ndi.binary_fill_holes(DAPI_binary)
    DAPI_clean = skimage.morphology.remove_small_objects(DAPI_filled_holes, min_size=20)
    #PI_dilation = skimage.morphology.binary_dilation(PI_binary)
    #PI_filled_holes = ndi.binary_fill_holes(PI_dilation)
    PI_clean = skimage.morphology.remove_small_objects(PI_binary, min_size=20)
    labeled_DAPI, _ = ndi.label(DAPI_clean)
    labeled_PI, _ = ndi.label(PI_clean)
    PI_props = measure.regionprops(labeled_PI)
    DAPI_props = measure.regionprops(labeled_DAPI)
    
    props_DAPI = pd.DataFrame(columns=['centroid x', 'centroid y', 'filled_area', 'area'])

    k = 1
    for props in DAPI_props:
    #Get the properties that I need for areas
    #Add them into a pandas dataframe that has the same number of rows as objects detected
        centroid = props.centroid
        centroid_x = centroid[0]
        centroid_y = centroid[1]
        fill_area = props.filled_area
        area = props.area
        props_DAPI.loc[k] = [centroid_x, centroid_y, fill_area, area]
        k = k + 1
        

    props_PI = pd.DataFrame(columns=['centroid x', 'centroid y', 'filled_area', 'area'])

    j = 1
    for props in PI_props:
    #Get the properties that I need for areas
    #Add them into a pandas dataframe that has the same number of rows as objects detected
        centroid = props.centroid
        centroid_x = centroid[0]
        centroid_y = centroid[1]
        fill_area = props.filled_area
        area = props.area
        props_PI.loc[j] = [centroid_x, centroid_y, fill_area, area]
        j = j + 1
    
    PI_overlap_sum = 0
    DAPI_pixels = 0
    i = 0
    PI_pixels = 0

    for ydim in range(0,512):
        for xdim in range(0,512):
            if labeled_DAPI[ydim][xdim] != 0:
                PI_overlap_sum = PI_overlap_sum + PI[ydim][xdim]
                DAPI_pixels = DAPI_pixels+1
                i = i + 1
                if labeled_PI[ydim][xdim] != 0:
                    PI_pixels = PI_pixels + 1
                    
    dead_count = 0
    live_count= 0

    for nuc in range(1, len(props_DAPI)+1):
        live = 0
        dead = 0
        for ydim in range(0,512):
            for xdim in range(0,512):
                if labeled_DAPI[ydim][xdim] == nuc:
                    if labeled_PI[ydim][xdim] == 0:
                        live = live+1
                    else:
                        dead = dead+1
        if dead/(live+dead) > 0.5:
            dead_count = dead_count+1
        else:
            live_count = live_count+1
    
    output_data = output_data.append({'Image':image, 'DAPI count':len(props_DAPI), 'PI count':len(props_PI), 'DAPI intensity':DAPI.sum(), 'PI intensity':PI.sum(), 'DAPI area':props_DAPI['area'].sum(), 'PI area':props_PI['area'].sum(), 'PI intensity/DAPI count':PI.sum()/len(props_DAPI), 'PI w/in DAPI area':PI_overlap_sum, 'PI w/in DAPI/DAPI count':PI_overlap_sum/len(props_DAPI), '%overlap PI/DAPI':PI_pixels/DAPI_pixels, '% cell death':dead_count/(dead_count+live_count)}, ignore_index=True)

In [ ]:
output_data

In [11]:
output_data.to_csv('1_25_20_PI_quantification.csv', encoding='utf-8', index=False)